In [1]:
import os, math, json, h5py, numpy as np, tensorflow as tf
from tensorflow.keras import models 
import qkeras
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

custom_objects = {
    'QDense': QDense,
    'QActivation': QActivation,
    'quantized_bits': quantized_bits,
    'quantized_relu': quantized_relu,
}

2025-08-13 00:12:39.555218: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:


# EDIT THESE
MODEL_H5 = '/home/users/dprimosc/L1DeepMET/outputs /models/25Aug7_JO_dense/model.h5'
DATA_H5 = '/home/users/dprimosc/L1DeepMET/data/25Jul8_140X_v0/TT_PU200/FP/140Xv0/perfNano_12509500_5.h5'  # optional; leave empty/None to use dummy inputs

print('TensorFlow:', tf.__version__)
print('Model path:', MODEL_H5)
print('Data path :', DATA_H5)

TensorFlow: 2.13.1
Model path: /home/users/dprimosc/L1DeepMET/outputs /models/25Aug7_JO_dense/model.h5
Data path : /home/users/dprimosc/L1DeepMET/data/25Jul8_140X_v0/TT_PU200/FP/140Xv0/perfNano_12509500_5.h5


In [4]:
model = models.load_model(MODEL_H5, custom_objects=custom_objects, compile=False)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_cat0 (InputLayer)     [(None, 128)]                0         []                            
                                                                                                  
 input_cat1 (InputLayer)     [(None, 128)]                0         []                            
                                                                                                  
 embedding0 (Embedding)      (None, 128, 2)               12        ['input_cat0[0][0]']          
                                                                                                  
 embedding1 (Embedding)      (None, 128, 2)               8         ['input_cat1[0][0]']          
                                                                                              

In [5]:
def input_names_and_shapes(model):
    names = [t.name.split(':')[0] for t in model.inputs]
    shapes = [tuple(int(d) if d is not None else -1 for d in t.shape[1:]) for t in model.inputs]
    dtypes = [t.dtype for t in model.inputs]
    return names, shapes, dtypes

def build_ordered_input_list(model, dict_by_name):
    in_names = [t.name.split(':')[0] for t in model.inputs]
    return [dict_by_name[n] for n in in_names]

def rand_like_shapes(shapes, dtypes, batch=16):
    arrs = []
    for shp, dt in zip(shapes, dtypes):
        shp = tuple(batch if d==-1 else d for d in shp)  # replace -1 with batch
        if dt.is_floating:
            arrs.append(np.random.randn(batch, *shp).astype(np.float32))
        else:
            # integer tokens if needed
            vocab = 16
            arrs.append(np.random.randint(0, vocab, size=(batch, *shp), dtype=np.int32))
    return arrs

def l2_norm_batch(arr):
    # arr: (B, ...); returns (B,) L2 norms
    flat = arr.reshape(arr.shape[0], -1)
    return np.linalg.norm(flat, axis=1)

def combined_input_diff_norm(x_list, x2_list):
    # L2 norm in input space combining all float inputs
    diffs = []
    for a, b in zip(x_list, x2_list):
        if a.dtype.kind in ('f',):  # floats only
            diffs.append(l2_norm_batch((b - a).astype(np.float32)))
    if not diffs:
        return np.zeros((x_list[0].shape[0],), dtype=np.float32)
    return np.sqrt(np.sum([d**2 for d in diffs], axis=0))

In [11]:
names, shapes, dtypes = input_names_and_shapes(model)

X_inputs = None
y = None

if DATA_H5 and os.path.exists(DATA_H5):
    with h5py.File(DATA_H5, 'r') as f:
        # Strategy:
        # 1) If datasets matching model input names exist, use them.
        # 2) Else if there's a single dataset 'X' and the model has a single input, use it.
        # 3) Else fallback to random.
        available = set(f.keys())
        by_name = {}
        matched = True
        for n, dt in zip(names, dtypes):
            if n in available:
                arr = f[n][...]
                if dt.is_floating:
                    arr = arr.astype(np.float32)
                else:
                    arr = arr.astype(np.int32)
                by_name[n] = arr
            else:
                matched = False
                break

        if matched:
            X_inputs = build_ordered_input_list(mdl, by_name)
        elif 'X' in available and len(names) == 1:
            arr = f['X'][...]
            X_inputs = [arr.astype(np.float32)]
        else:
            print('Could not match datasets by model input names; using random dummy inputs.')
            X_inputs = rand_like_shapes(shapes, dtypes, batch=32)

        # Optional targets
        if 'Y' in available:
            y = f['Y'][...].astype(np.float32)
else:
    print('DATA_H5 not provided/found; creating random dummy inputs.')
    X_inputs = rand_like_shapes(shapes, dtypes, batch=32)
    y = None

# Keep a small eval slice
X_eval = [a[:32] for a in X_inputs]
print('Input names:', names)
print('Eval shapes:', [a.shape for a in X_eval])

Could not match datasets by model input names; using random dummy inputs.
Input names: ['input_cont', 'input_pxpy', 'input_cat0', 'input_cat1']
Eval shapes: [(32, 128, 4), (32, 128, 2), (32, 128), (32, 128)]


In [13]:
def empirical_lipschitz(model, inputs, eps=1e-3, trials=5, seed=123):
    rng = np.random.default_rng(seed)
    B = inputs[0].shape[0]
    y_ref = model.predict(inputs, verbose=0)
    y_ref = np.array(y_ref)  # ensure ndarray

    ratios_all = []
    for t in range(trials):
        pert = []
        for arr in inputs:
            if arr.dtype.kind in ('f',):
                noise = rng.standard_normal(arr.shape, dtype=np.float32) * eps
                pert.append(arr + noise)
            else:
                pert.append(arr)  # keep categorical/indices fixed
        y_pert = model.predict(pert, verbose=0)
        y_pert = np.array(y_pert)

        dy = y_pert - y_ref
        dy_norm = l2_norm_batch(dy.astype(np.float32))
        dx_norm = combined_input_diff_norm(inputs, pert)
        # avoid div by zero
        dx_norm = np.where(dx_norm == 0, 1e-12, dx_norm)
        ratios = dy_norm / dx_norm
        ratios_all.append(ratios)

    ratios_all = np.stack(ratios_all, axis=0)  # (trials, B)
    stats = {
        'mean': float(np.mean(ratios_all)),
        'p95': float(np.percentile(ratios_all, 95)),
        'max': float(np.max(ratios_all)),
    }
    return stats, ratios_all

emp_stats, emp_ratios = empirical_lipschitz(model, X_eval, eps=1e-3, trials=5)
print('Empirical Lipschitz (perturbation) stats:', emp_stats)

InvalidArgumentError: Graph execution error:

Detected at node 'model/embedding1/embedding_lookup' defined at (most recent call last):
    File "/home/users/dprimosc/micromamba/envs/l1metml/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/users/dprimosc/micromamba/envs/l1metml/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/users/dprimosc/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/users/dprimosc/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/users/dprimosc/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/users/dprimosc/.local/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_